In [186]:
import numpy as np
import pandas as pd
import plotly.express as px
from tqdm import tqdm
from sklearn.metrics import classification_report

## ID Boss Sentiment

In [187]:
# id_logs = pd.read_csv("../results/2023-09-12_01-38-43_imdb_rotten_tomatoes_Kyle1668-imdb-bert-base-uncased/Kyle1668-imdb-bert-base-uncased-imdb_rotten_tomatoes-test-static-No Adaptation-None-TrimExemplars=NA-inference-logs.csv")
# id_logs = pd.read_csv("../results/2023-09-12_01-35-04_boss_sentiment_Kyle1668-boss-sentiment-bert-base-uncased/Kyle1668-boss-sentiment-bert-base-uncased-boss_sentiment-validation-static-No Adaptation-None-TrimExemplars=NA-inference-logs.csv")
id_logs = pd.read_csv("../results/2023-09-12_02-56-21_ag_news_twitter_Kyle1668-ag-news-bert-base-uncased/Kyle1668-ag-news-bert-base-uncased-ag_news_twitter-test-static-No Adaptation-None-TrimExemplars=NA-inference-logs.csv")


display(id_logs.head())
display(id_logs.shape)

,entropy,probs,latency,input,judgment,prompt,label
0,0.000118,"tensor([4.2235e-06, 6.3471e-07, 9.9999e-01, 3....",0.008440,Unions express disappointment in talks with Fe...,2,"['Task: Given the news text, classify whether ...",2
1,0.000107,"tensor([3.0789e-06, 1.0066e-06, 3.6746e-06, 9....",0.006947,🚀👨‍🚀 #SpaceRace update: Second private team to...,3,"['Task: Given the news text, classify whether ...",3
2,0.000248,"tensor([4.2797e-06, 4.3314e-07, 1.5252e-05, 9....",0.006886,"""Chemistry researcher's Ky. startup wins grant...",3,"['Task: Given the news text, classify whether ...",3
3,0.000103,"tensor([2.4275e-06, 1.6679e-06, 3.3567e-06, 9....",0.006855,"🔥💨🌲Prediction Unit helps forecast wildfires, s...",3,"['Task: Given the news text, classify whether ...",3
4,0.000105,"tensor([3.3087e-06, 1.4950e-06, 2.7821e-06, 9....",0.006697,"""California takes on smog w emissions rules fo...",3,"['Task: Given the news text, classify whether ...",3


(7600, 7)

In [188]:
id_logs["correct"] = id_logs["judgment"] == id_logs["label"]
id_logs["label"] = id_logs["label"].astype(str)
id_correct_summary = id_logs.groupby(["correct"])["entropy"].describe()

id_entropy_scatter = px.scatter(id_logs, y="entropy", color="label", title="Boss Sentiment ID Entropy by Correct", log_y=True, hover_data=["label"])

print("Boss Sentiment In-Distribution Entropy Analysis")
display(id_correct_summary)
id_entropy_scatter.show()

Boss Sentiment In-Distribution Entropy Analysis


,count,mean,std,min,25%,50%,75%,max
correct,,,,,,,,
False,869.0,0.048413,0.145611,0.000014,0.000123,0.000260,0.002862,0.873378
True,6731.0,0.004962,0.046126,0.000014,0.000054,0.000104,0.000124,0.693227


## OOD Boss Sentiment

In [189]:
ood_with_icr_logs = pd.read_csv("../results/2023-09-12_02-56-21_ag_news_twitter_Kyle1668-ag-news-bert-base-uncased/Kyle1668-ag-news-bert-base-uncased-ag_news_twitter-topk_nearest-test-stabilityai-StableBeluga-7B-8-style_inference_log.csv")
# ood_with_icr_logs = pd.read_csv("../results/2023-09-12_01-50-53_boss_sentiment_Kyle1668-boss-sentiment-bert-base-uncased/Kyle1668-boss-sentiment-bert-base-uncased-boss_sentiment-sst5-random-sst5-stabilityai-StableBeluga-7B-16-style_inference_log.csv")
# ood_with_icr_logs = pd.read_csv("../results/2023-09-12_01-38-43_imdb_rotten_tomatoes_Kyle1668-imdb-bert-base-uncased/Kyle1668-imdb-bert-base-uncased-imdb_rotten_tomatoes-topk_nearest-test-stabilityai-StableBeluga-7B-8-style_inference_log.csv")
display(ood_with_icr_logs.head())
display(ood_with_icr_logs.shape)

,entropy,probs,latency,input,original_input,style prompt,mean exemplar distance,judgment,prompt,label,original judgment,original entropy,entropy decrease,entropy decreased,outcome
0,0.338919,"tensor([4.7107e-04, 9.6917e-05, 8.9537e-01, 1....",3.434628,['In response to recent negotiations between f...,Unions express disappointment in talks with Fe...,### System:\n### Instructions ###\nThe assista...,0.599328,2,"['Task: Given the news text, classify whether ...",2,2,0.000118,-0.338802,False,Unchanged Correct
1,0.000113,"tensor([2.5583e-06, 1.2354e-06, 4.4956e-06, 9....",4.233715,['Now parphrase this tweet about the second pr...,🚀👨‍🚀 #SpaceRace update: Second private team to...,### System:\n### Instructions ###\nThe assista...,0.665975,3,"['Task: Given the news text, classify whether ...",3,3,0.000107,-0.000007,False,Unchanged Correct
2,0.005346,"tensor([8.5618e-06, 5.5391e-06, 6.1540e-04, 9....",4.036771,['Kentucky chemistry start-up receives grant f...,"""Chemistry researcher's Ky. startup wins grant...",### System:\n### Instructions ###\nThe assista...,0.570652,3,"['Task: Given the news text, classify whether ...",3,3,0.000248,-0.005097,False,Unchanged Correct
3,0.000111,"tensor([2.2369e-06, 2.7144e-06, 3.1130e-06, 9....",5.026832,['Now rephrases the given statement using diff...,"🔥💨🌲Prediction Unit helps forecast wildfires, s...",### System:\n### Instructions ###\nThe assista...,0.487785,3,"['Task: Given the news text, classify whether ...",3,3,0.000103,-0.000008,False,Unchanged Correct
4,0.000124,"tensor([3.4183e-06, 1.1969e-06, 4.5292e-06, 9....",3.839096,['In response to rising concerns about air qua...,"""California takes on smog w emissions rules fo...",### System:\n### Instructions ###\nThe assista...,0.677534,3,"['Task: Given the news text, classify whether ...",3,3,0.000105,-0.000019,False,Unchanged Correct


(7600, 15)

In [190]:
ood_with_icr_logs.groupby("outcome")[["entropy", "original entropy"]].describe()

entropy                                                    \
                    count      mean       std       min       25%       50%   
outcome                                                                       
New Correct         174.0  0.196792  0.226598  0.000101  0.012000  0.092994   
New Mistake         603.0  0.308358  0.237208  0.000167  0.121219  0.282899   
Unchanged Correct  6128.0  0.047345  0.122897  0.000014  0.000111  0.000275   
Unfixed Mistake     628.0  0.098224  0.202380  0.000016  0.000180  0.003679   

                                      original entropy                      \
                        75%       max            count      mean       std   
outcome                                                                      
New Correct        0.338776  1.071353            174.0  0.114030  0.216308   
New Mistake        0.451443  1.091039            603.0  0.022747  0.094551   
Unchanged Correct  0.006767  1.042588           6128.0  0.003211  0.037741   
Unfixed Mistake    0.057447  1.056889            628.0  0.027661  0.107528   

                                                                     
                        min       25%       50%       75%       max  
outcome                                                              
New Correct        0.000018  0.000315  0.002757  0.078583  0.873378  
New Mistake        0.000014  0.000080  0.000123  0.000293  0.693227  
Unchanged Correct  0.000014  0.000032  0.000104  0.000120  0.690946  
Unfixed Mistake    0.000014  0.000115  0.000181  0.000736  0.763525

In [191]:
# AG News: Combine Labels
# ood_with_icr_logs["label"] = ood_with_icr_logs["label"].apply(lambda x: 3 if x == 2 else x)
# ood_with_icr_logs["judgment"] = ood_with_icr_logs["judgment"].apply(lambda x: 3 if x == 2 else x)
# ood_with_icr_logs["original judgment"] = ood_with_icr_logs["original judgment"].apply(lambda x: 3 if x == 2 else x)

In [192]:
print("Performance Without Rewritten Judgments")
print(classification_report(ood_with_icr_logs["label"], ood_with_icr_logs["original judgment"], digits=4))

print("Performance Using Rewritten Judgments")
print(classification_report(ood_with_icr_logs["label"], ood_with_icr_logs["judgment"], digits=4))

entropy_threshold = 0.000124
print(f"Performance Using Rewritten Judgments at Entropy Thresholds = {entropy_threshold}")
hybrid_judgments = ood_with_icr_logs.apply(lambda row: row["original judgment"] if row["original entropy"] < entropy_threshold else row["judgment"], axis=1)
print(classification_report(ood_with_icr_logs["label"], hybrid_judgments, digits=4))

rewriter_call_count = (ood_with_icr_logs["original entropy"] >= entropy_threshold).sum()
print(f"Rewriter Call Count: {rewriter_call_count} - Rate: {100 * rewriter_call_count / len(ood_with_icr_logs)}%")

Performance Without Rewritten Judgments
              precision    recall  f1-score   support

           0     0.9259    0.8547    0.8889      1900
           1     0.9654    0.9684    0.9669      1900
           2     0.9095    0.7621    0.8293      1900
           3     0.7747    0.9574    0.8564      1900

    accuracy                         0.8857      7600
   macro avg     0.8939    0.8857    0.8854      7600
weighted avg     0.8939    0.8857    0.8854      7600

Performance Using Rewritten Judgments
              precision    recall  f1-score   support

           0     0.9697    0.7253    0.8299      1900
           1     0.9635    0.9453    0.9543      1900
           2     0.9138    0.6753    0.7766      1900
           3     0.6338    0.9711    0.7670      1900

    accuracy                         0.8292      7600
   macro avg     0.8702    0.8292    0.8320      7600
weighted avg     0.8702    0.8292    0.8320      7600

Performance Using Rewritten Judgments at Entropy Thr

In [198]:
def get_threshold_f1(threshold):
    threshold_judgments = ood_with_icr_logs.apply(lambda row: row["original judgment"] if row["original entropy"] < threshold else row["judgment"], axis=1)
    report = classification_report(ood_with_icr_logs["label"], threshold_judgments, digits=4, output_dict=True)
    llm_call_count = (ood_with_icr_logs["original entropy"] >= threshold).sum()
    llm_call_rate = 100 * llm_call_count / len(ood_with_icr_logs) 
    return 100 * report["macro avg"]["f1-score"], llm_call_rate

thresholds = np.arange(0, 1, 0.00005)
threshold_scores = []
threshold_rewrite_rates = []

for t in tqdm(thresholds):
    t_perf, t_rate = get_threshold_f1(t)
    threshold_scores.append(t_perf)
    threshold_rewrite_rates.append(t_rate)
    
    
thresholds_frame = pd.DataFrame({"threshold": thresholds, "f1": threshold_scores, "rewrite_rate": threshold_rewrite_rates})
thresholds_frame

100%|██████████| 20000/20000 [15:25<00:00, 21.62it/s]


,threshold,f1,rewrite_rate
0,0.00000,83.195029,100.000000
1,0.00005,83.838660,77.302632
2,0.00010,86.008251,58.526316
3,0.00015,87.626862,22.223684
4,0.00020,87.867588,16.605263
...,...,...,...
19995,0.99975,88.537761,0.000000
19996,0.99980,88.537761,0.000000
19997,0.99985,88.537761,0.000000
19998,0.99990,88.537761,0.000000


In [199]:
thresholds_frame[thresholds_frame["rewrite_rate"].round() == 6]

,threshold,f1,rewrite_rate
37,0.00185,88.777282,6.500000
38,0.00190,88.788632,6.460526
39,0.00195,88.802317,6.381579
40,0.00200,88.802317,6.342105
41,0.00205,88.814342,6.250000
42,0.00210,88.826982,6.171053
43,0.00215,88.826982,6.118421
44,0.00220,88.801379,6.052632
45,0.00225,88.801379,5.986842
46,0.00230,88.788472,5.947368


In [200]:
figure = px.line(thresholds_frame,
        x="rewrite_rate",
        y="f1",
        title="IMDB --> Rotten Tomatoes Rewrite-Rate-F1 Curve", 
        # log_x=True,
        # log_y=True,
        )

figure.show()